<a id="top"></a>
# ULLYSES Data Download Tutorial

***

## Learning Goals

By the end of this tutorial, you will:

- Know how to use Astroquery to download ULLYSES HLSPs
- Be able to use ullyses-utils.select_pids to get PIDs for desired subsets of ULLYSES programs, and know how to download those datasets
- Understand where to find and how to download the files that went into a ULLYSES HLSP


## Table of Contents
**0. [Introduction](#introduction)**

**1. [Downloading HLSP Files Using Astroquery](#astroquery)**

**2. [Using ullyses_utils.select_pids](#selectpids)**

**3. [Downloading HLSP Constiuent Raw Data Files](#constiuent)**

## Introduction
The Hubble Space Telescope’s (HST) Ultraviolet Legacy Library of Young Stars as Essential Standards ([ULLYSES](https://ullyses.stsci.edu/index.html) program has devoted approximately 1,000 HST orbits to the production of an ultraviolet spectroscopic library of young high- and low-mass stars in the local universe. This Director’s Discretionary program has been designed to take advantage of HST’s unique UV capabilities, as both high- and low-mass stars feature different complex UV emission processes that strongly impact their surroundings, but are difficult to model. The UV emission from star formation is central to a wide range of vital astrophysical issues, ranging from cosmic reionization to the formation of planets.

The ULLYSES program has uniformly sampled the fundamental astrophysical parameter space for each mass regime — including spectral type, luminosity class, and metallicity for massive OB stars (in the Magellanic Clouds and two other lower-metallicity nearby galaxies) and the mass, age, and disk accretion rate for low-mass T Tauri stars (in eight young Galactic associations). The data were gathered over a three-year period, from Cycle 27 through Cycle 29 (2020-2022).

The ULLYSES team produces several types of High Level Science Products (HLSPs). Products are made using both archival data and new HST observations obtained through the ULLYSES program. Data products are available from the [ULLYSES website](https://ullyses.stsci.edu/ullyses-download.html) (HLSPs and contributing data), the [MAST Data Discovery Portal](https://mast.stsci.edu/) (HLSPs and contributing data), directly as a High-Level Science Product collection using the [DOI](https://archive.stsci.edu/hlsp/ullyses) (HLSPs only), or using the Python package [Astroquery](https://astroquery.readthedocs.io/en/latest/) to search for and download files from Python scripts you write.

This notebook will guide users through downloading HLSPs and raw ULLYSES data through various means using Astroquery.

### Imports
<!-- - *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files -->
- *astropy.table Table* for creating tidy tables of the data
<!-- - *matplotlib.pyplot* for plotting data -->
- *Path* to create product and data directories
- *shutil* to perform directory and file operations
- *glob* to work with multiple files in our directories
<!-- - *os* to interact with the operating system -->
<!-- - *pandas* to support data analysis -->
- *astroquery* to download HLSPs and raw data files

In [117]:
# %matplotlib inline
# import numpy as np
# from astropy.io import fits
# from astropy.wcs import WCS
from astropy.table import Table
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize']=10,6
# plt.style.use('seaborn-v0_8-notebook')
# import os
# from pathlib import Path
import shutil
import glob
from astroquery.mast import Observations

# add some text to intro about how to install ullyses and ullyses utils!!

from ullyses_utils.select_pids import select_all_pids, select_pids

***

<a id="astroquery"></a>
# Downloading HLSP Files Using Astroquery

The `Observations` Class in Astroquery.mast has several different useful functions for searching and downloading data products from MAST. We can use the `query_criteria` function to first search out all ULLYSES HLSP data that is available on MAST using the search criteria `provenance_name='ULLYSES'` as follows:

In [114]:
search = Observations.query_criteria(provenance_name='ULLYSES')

The above cell returns `search` which is an Astropy.Table that holds our search results. We can print the results next:

In [115]:
# This down-selects the number of columns printed to just a few, which makes it easier to read
search.pprint_include_names = ('target_name', 's_ra', 's_dec', 'proposal_id', 'instrument_name', 'filters')

search.pprint()

# To print all columns use:
# search.pprint_all()

# Or to print a single column use:
# search['target_name'].pprint()

                 instrument_name                                filters                     target_name              s_ra               s_dec                  proposal_id          
-------------------------------------------------- ---------------------------------- ----------------------- ------------------ ------------------- -------------------------------
                                           COS/FUV                        G130M;G160M                2DFS-163       9.2426666667      -73.3925519167                           16100
                                           COS/FUV                        G130M;G160M               2DFS-2266      16.8094758333      -72.2298698889                           16370
                           COS/FUV | STIS/FUV-MAMA                        E140M;G130M               2DFS-2553        17.34146875      -73.2616487222                     15629_16803
                                           COS/FUV                        G130M;G160M          

As you can see, there are several results returned for the initial search. We can down-select our search by adding the name of the target we want, using the criteria `target_name='V505-ORI'`.

In [116]:
search_target = Observations.query_criteria(target_name='V505-ORI', provenance_name='ULLYSES')

Next, we use the function `get_product_list` with the input of the observation IDs that are returned from the query in the previous cell to get a table of all the data products that are available in MAST for these observations.

In [125]:
data_products = Observations.get_product_list(search_target['obsid'])
print(data_products['productFilename'])

                        productFilename                        
---------------------------------------------------------------
hlsp_ullyses_hst_cos-stis_v505-ori_uv-opt_dr6_preview-spec.fits
       hlsp_ullyses_hst_cos_v505-ori_g130m-g160m_dr6_cspec.fits
             hlsp_ullyses_hst_cos_v505-ori_g130m_dr6_cspec.fits
             hlsp_ullyses_hst_cos_v505-ori_g160m_dr6_cspec.fits
hlsp_ullyses_hst_stis_v505-ori_g230l-g430l-g750l_dr6_cspec.fits
            hlsp_ullyses_hst_stis_v505-ori_g230l_dr6_cspec.fits
             hlsp_ullyses_hst_stis_v505-ori_g230l_dr6_spec.fits
             hlsp_ullyses_hst_stis_v505-ori_g230l_dr6_spec.yaml
            hlsp_ullyses_hst_stis_v505-ori_g430l_dr6_cspec.fits
             hlsp_ullyses_hst_stis_v505-ori_g430l_dr6_spec.fits
             hlsp_ullyses_hst_stis_v505-ori_g430l_dr6_spec.yaml
            hlsp_ullyses_hst_stis_v505-ori_g750l_dr6_cspec.fits
             hlsp_ullyses_hst_stis_v505-ori_g750l_dr6_spec.fits
             hlsp_ullyses_hst_stis_v505-

Then, we can use the function `download_products` to download all the files from our search. The user can specify where to download the data products with `download_dir`. Note that we specify `extension=['fits']`; there are also `yaml` files that may be downloaded. You can add those by adding 'yaml' to the extension list. Note also that we must specify `obs_collection=['HLSP']` so that we only get the high level science products, and not the constiuent data. If you want both, you can delete the `obs_collection` specification altogether. 

In [144]:
output_dir = './v505-ori/'
Observations.download_products(data_products, download_dir=output_dir, extension=['fits'], obs_collection=['HLSP'])

Local Path,Status,Message,URL
str151,str8,object,object
./v505-ori/mastDownload/HLSP/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m_dr6_cspec.fits,COMPLETE,None,None
./v505-ori/mastDownload/HLSP/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m_dr6_cspec.fits,COMPLETE,None,None
./v505-ori/mastDownload/HLSP/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g160m_dr6_cspec.fits,COMPLETE,None,None
./v505-ori/mastDownload/HLSP/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g185m_dr6_cspec.fits,COMPLETE,None,None
./v505-ori/mastDownload/HLSP/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m/hlsp_ullyses_hst_cos_ngc1818-rob-d1_g225m_dr6_cspec.fits,COMPLETE,None,None


All ULLYSES HLSP data products are now downloaded to your `download_dir` in the sub-directories starting with `mastDownload`. We can move these into a better folder structure using the following function.

In [149]:
def mv_downloads(output_path):
    # Specify the path to where the downloads were placed
    mast_path = os.path.join(output_path, 'mastDownload/')
    
    # Get a list of all obs_id folders. Each folder contains the FITS files
    obs_id_dirs = glob.glob(os.path.join(mast_path, '*', '*'))
    
    # Iterate through each of the sub-folders to change the path of each FITS file
    for subdir in obs_id_dirs: 
    
        # Get a list of all FITS files in the current ./mastDownload/*/<obs_id> folder
        sub_files = glob.glob(subdir + '/*fits')
    
        # Iterating through each of these files to change their path individually:
        # We will be moving them to the top level, ./v505-ori/ in this example
        for file in sub_files: 
            original_path = os.path.join(obs_id_path, file)
            new_path = os.path.join(output_path, file.split('/')[-1])
            shutil.move(file, new_path)
    
    # Last, remove the mastDownload directory
    shutil.rmtree(os.path.join(output_path, 'mastDownload/'))

mv_downloads(output_dir)

Now, all your HLSP files should be nice and tidy in your output directory!

<a id="selectpids"></a>
# Using ullyses_utils.select_pids

The `select_pids` script in `ullyses_utils` has two functions that allow users to select the PIDS of both ULLYSES-orbserved and archival datasets for specific sub-sets of data. For example, one can select a specific region or target type. We will show some examples of these selections next.

The function `select_all_pids()` is useful for selecting out different target types across many regions. We can see more information in the functions docstring by typing the following:

In [7]:
select_all_pids?

Signature:
select_all_pids(
    massive=False,
    tts=False,
    extra=True,
    monitoring=True,
    single_list=True,
)
Docstring:
Use this function if you want to select more than one region's PIDs at once.
The logic behind writing this is:
1) if you want massive stars (LMC, SMC, lowz), specify massive=True. If you
do not want the "extra" stars included in the ULLYSES sample (i.e. stars
that were added after the original sample was created to fill the
luminosity/class bins), then specify extra=False.
  -> Original sample + Extra sample: pids = select_all_pids(massive=True)
  -> Original sample only: pids = select_all_pids(massive=True, extra=False)
2) if you want all regions of the t-tauri star sample, including the monitoring
stars, specify tts=True. If you do not want the monitoring stars in the sample
then specify monitoring=False.
  -> All TTS: pids = select_all_pids(tts=True)
  -> All TTS but the monitoring: pids = select_all_pids(tts=True, monitoring=False)
3) You want the en

The function `select_pids` gives all the PIDs of a certain region regardless of target type. See the options in the docstring below:

In [8]:
select_pids?

Signature: select_pids(selected_region, single_list=True)
Docstring:
This function returns pids given a region. "Extra" regions are separated
out as these include datasets that were not originally in the ULLYSES sample.
The TTS regions are separated by their host cluster. Low metallicity stars
are separated by imaging and spectroscopic observations.

Inputs
------
selected_region : str
    Accepted regions:
      - "smc-extra", "smc", "lmc-extra", "lmc",
        "lowz-extra", "lowz-image", "lowz",
        "monitoring_tts",
        "cha i", "cra", "eps cha", "eta cha", "lupus", "ori ob", "sigma ori",
          "taurus", "twa", "lower centaurus", "upper scorpius", "other", "lambda ori"
single_list : Boolean
  True by default. If true, a list of pids for the specified region is returned.
  If false, two lists of pids are returned; one is only the ULLYSES observed
  PIDs (those the team made Phase 2s for from 2020-2023), and then other is
  only the archival data used in the HLSP creation 

Example 1: select all ULLYSES PIDs

In [9]:
all_ullyses_pids = select_all_pids()
print(all_ullyses_pids)

['11487', '11489', '11531', '11533', '11608', '11616', '11623', '11625', '11660', '11692', '11828', '11997', '12010', '12036', '12161', '12199', '12218', '12315', '12425', '12581', '12587', '12717', '12805', '12867', '12876', '12907', '12978', '12996', '13004', '13032', '13122', '13363', '13372', '13373', '13522', '13714', '13766', '13775', '13778', '13780', '13781', '13806', '13931', '13969', '14048', '14081', '14177', '14193', '14245', '14246', '14437', '14469', '14476', '14503', '14604', '14675', '14683', '14690', '14703', '14712', '14855', '14909', '15070', '15128', '15156', '15165', '15204', '15210', '15310', '15366', '15385', '15536', '15629', '15774', '15824', '15837', '15880', '15921', '15967', '16010', '16090', '16091', '16092', '16093', '16094', '16095', '16096', '16097', '16098', '16099', '16100', '16101', '16102', '16103', '16104', '16107', '16108', '16109', '16110', '16113', '16114', '16115', '16230', '16272', '16290', '16304', '16364', '16365', '16366', '16367', '16368', 

Example 2: select all SMC and LMC targets and their extras

In [76]:
all_smc_lmc_pids = select_pids('smc') + select_pids('lmc')
print(all_smc_lmc_pids)

['16098', '16099', '16100', '16101', '16102', '16103', '16368', '16370', '16371', '16372', '16373', '16375', '16802', '16803', '16805', '16806', '16807', '16808', '16809', '17295', '7437', '7480', '8145', '8566', '9094', '9116', '9383', '9434', '11487', '11489', '11623', '11625', '11997', '12010', '12425', '12581', '12717', '12805', '13004', '13122', '13373', '13522', '13778', '13931', '13969', '14081', '14437', '14476', '14503', '14712', '14855', '14909', '15366', '15385', '15536', '15629', '15774', '15837', '16230', '16090', '16091', '16092', '16093', '16094', '16095', '16096', '16097', '16364', '16365', '16366', '16367', '16369', '16374', '16810', '16811', '16812', '16813', '16814', '16815', '16816', '16817', '16818', '16819', '16820', '16821', '16822', '16823', '16824', '16825', '16826', '17296', '7299', '7392', '8320', '8662', '9434', '9757', '12218', '12581', '12978', '14675', '14683', '15629', '15824', '16304', '16230']


Example 3: select all massive stars without the extras, and return a list separated by ULLYSES PIDs vs Archival PIDS

In [13]:
massive_pids = select_all_pids(massive=True, extra=False, single_list=False)
print(massive_pids['ULLYSES'])
print(massive_pids['ARCHIVAL'])

['16090', '16091', '16092', '16093', '16094', '16095', '16096', '16097', '16098', '16099', '16100', '16101', '16102', '16103', '16104', '16364', '16365', '16366', '16367', '16368', '16369', '16370', '16371', '16372', '16373', '16374', '16375', '16511', '16802', '16803', '16805', '16806', '16807', '16808', '16809', '16810', '16811', '16812', '16813', '16814', '16815', '16816', '16817', '16818', '16819', '16820', '16821', '16822', '16823', '16824', '16825', '16826', '16930', '17295', '17296']
['11487', '11489', '11623', '11625', '11997', '12010', '12218', '12425', '12581', '12717', '12805', '12978', '13004', '13122', '13373', '13522', '13778', '13931', '13969', '14081', '14437', '14476', '14503', '14675', '14683', '14712', '14855', '14909', '15366', '15385', '15536', '15629', '15774', '15824', '15837', '16230', '16304', '7299', '7392', '7437', '7480', '8145', '8320', '8566', '8662', '9094', '9116', '9383', '9434', '9757']


We can again use Astroquery to download the HLSPs from these selected PIDs. The following example is for our second selection on all SMC and LMC targets and their extras.

In [133]:
search_pids = Observations.query_criteria(proposal_id=all_smc_lmc_pids, provenance_name='ULLYSES')
search_pids.pprint()

data_products_pids = Observations.get_product_list(search_pids['obsid'])

output_dir_smc_lmc = './all_smc_lmc'

Observations.download_products(data_products_pids, download_dir=output_dir_smc_lmc, extension=['fits'], obs_collection=['HLSP'])

mv_downloads(output_dir_smc_lmc)

intentType obs_collection provenance_name          instrument_name          project         filters         wavelength_region  target_name   ... jpegURL dataURL dataRights mtFlag srcDen   obsid     objID  
---------- -------------- --------------- --------------------------------- ------- ----------------------- ----------------- -------------- ... ------- ------- ---------- ------ ------ --------- ---------
   science           HLSP         ULLYSES                           COS/FUV     HST             G130M;G160M                UV       2DFS-163 ...      --      --     PUBLIC  False    nan 118405126 219472731
   science           HLSP         ULLYSES                           COS/FUV     HST             G130M;G160M                UV      2DFS-2266 ...      --      --     PUBLIC  False    nan 118405127 219472738
   science           HLSP         ULLYSES                           COS/FUV     HST             G130M;G160M                UV      2DFS-3689 ...      --      --     PUBLIC  Fal

<a id="constiuent"></a>
# Downloading HLSP Constiuent Raw Data Files

For the last example, we'll show how to download the raw data files that made up a HLSP. We will use one of the data sets that we downloaded in the last example here. Our HLSP will be the file `hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m_dr6_cspec.fits`, which is a level 4 product for the target NGC1818-ROB-D1. We can look at the provenance table in this file to find the constiuent spectra.

In [136]:
hlsp_file_path = os.path.join(output_dir_smc_lmc, 'hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m_dr6_cspec.fits')

In [137]:
prov_table = Table(fits.getdata(hlsp_file_path, ext=2))
prov_table.pprint_include_names = ('FILENAME', 'PROPOSID', 'INSTRUMENT', 'DETECTOR', 'DISPERSER', 'APERTURE', 'XPOSURE')
prov_table.pprint()

     FILENAME      PROPOSID INSTRUMENT DETECTOR DISPERSER APERTURE XPOSURE
------------------ -------- ---------- -------- --------- -------- -------
lcyid1acq_x1d.fits    14081        COS      FUV     G130M      PSA 521.024
lcyid1anq_x1d.fits    14081        COS      FUV     G130M      PSA 570.016
lcyid1agq_x1d.fits    14081        COS      FUV     G130M      PSA 570.016
lcyid1ayq_x1d.fits    14081        COS      FUV     G130M      PSA 570.048
lcyid1aaq_x1d.fits    14081        COS      FUV     G130M      PSA 521.024
lcyid1arq_x1d.fits    14081        COS      FUV     G130M      PSA 570.016
lcyid1aeq_x1d.fits    14081        COS      FUV     G130M      PSA 521.024
lcyid1a8q_x1d.fits    14081        COS      FUV     G130M      PSA 521.024
lcyid1crq_x1d.fits    14081        COS      FUV     G160M      PSA  2003.2
lcyid1cjq_x1d.fits    14081        COS      FUV     G160M      PSA  2003.2
lcyid1b6q_x1d.fits    14081        COS      FUV     G160M      PSA  1735.2
lcyid1c9q_x1d.fits    140

In [138]:
# Make a list that is just the constiuent files from the provenance data
# We will use this for comparison later
spectra_filenames = prov_table['FILENAME']

Next, we'll run a search using Astroquery on the target name and specifying the provenance name as ULLYSES. Then, we can see the data products associated with this search.

In [139]:
search_target = Observations.query_criteria(target_name='ngc1818-rob-d1', provenance_name='ULLYSES')

data_products = Observations.get_product_list(search_target['obsid'])
print(data_products['productFilename'])

                          productFilename                           
--------------------------------------------------------------------
hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m-g160m-g185m_dr6_cspec.fits
            hlsp_ullyses_hst_cos_ngc1818-rob-d1_g130m_dr6_cspec.fits
            hlsp_ullyses_hst_cos_ngc1818-rob-d1_g160m_dr6_cspec.fits
            hlsp_ullyses_hst_cos_ngc1818-rob-d1_g185m_dr6_cspec.fits
            hlsp_ullyses_hst_cos_ngc1818-rob-d1_g225m_dr6_cspec.fits
                                                  lcyid1010_asn.fits
                                                  lcyid1010_jif.fits
                                                  lcyid1010_jit.fits
                                                  lcyid1010_trl.fits
                                              lcyid1010_x1dsum1.fits
                                              lcyid1010_x1dsum2.fits
                                              lcyid1010_x1dsum3.fits
                                  

Let's use a for-loop to compare the constiuent filenames from the provenance table to the data products returned from our query. We'll only save the observation IDs of the matching data products for later downloading.

In [140]:
to_download = []
for assocfile, obsid in zip(data_products['productFilename'], data_products['obsID']):
    if assocfile in filenames:
        to_download.append(obsid)

Last, we can download the products that we saved in `to_download`. In the call to `download_products` in the following cell, not that we must specify the `obs_collection=['HST']`, which ensures we will not download any HLSPs, as well as `productSubGroupDescription=['X1D', 'SX1']`, otherwise all the other COS or STIS data products will be downloaded as well. One may change these parameters if more data products are desired.

In [150]:
output_dir = './ngc1818-rob-d1/'
Observations.download_products(to_download, download_dir=output_dir, extension=['fits'], obs_collection=['HST'], productSubGroupDescription=['X1D', 'SX1'])
mv_downloads(output_dir)

INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyid1a8q/lcyid1a8q_x1d.fits with expected size 3389760. [astroquery.query]
INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyid1aaq/lcyid1aaq_x1d.fits with expected size 3389760. [astroquery.query]
INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyid1acq/lcyid1acq_x1d.fits with expected size 3389760. [astroquery.query]
INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyid1aeq/lcyid1aeq_x1d.fits with expected size 3389760. [astroquery.query]
INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyid1agq/lcyid1agq_x1d.fits with expected size 3389760. [astroquery.query]
INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyid1anq/lcyid1anq_x1d.fits with expected size 3389760. [astroquery.query]
INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyid1arq/lcyid1arq_x1d.fits with expected size 3389760. [astroquery.query]
INFO: Found cached file ./ngc1818-rob-d1/mastDownload/HST/lcyi

#### Now, go forth and download and make beautiful discoveries with our ULLYSES products!

***

## Additional Resources

- [ULLYSES](https://ullyses.stsci.edu)
- [MAST API](https://mast.stsci.edu/api/v0/index.html)

## About this Notebook
For support, contact us at the [ULLYSES Helpdesk](https://stsci.service-now.com/hst?id=sc_cat_item&sys_id=a3b8ec5edbb7985033b55dd5ce961990&sysparm_category=ac85189bdb4683c033b55dd5ce96199c).

**Author:**  Elaine M Frazer

**Updated On:** 2023-12-07

## Citations
* See the [ULLYSES website](https://ullyses.stsci.edu/ullyses-cite.html) for citation guidelines.

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 